<a href="https://colab.research.google.com/github/marcgym2/MachineLearning/blob/main/Tarea5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tarea en clase (2 puntos)
- Investigar sobre algún otro algoritmo no supervisado que pueda usarse en tu código. Da un panorama sobre el modelo matemático que emplea y explica por qué conviene aplicarlo a tus datos. Algunos modelos que no se vieron en clase son Affinity Propagation, BIRCH, DBSCAN, Mean Shift, Nearest Neighbors, OPTICS, Spectral Clustering, TSNE, entre otros
- Investigar otras estrategias para determinar número de grupos en estos algoritmos (como los índices de Calinski-Harabasz o de Davies-Bouldin), elegir la más adecuada al método que elegiste


# DBSCAN:  (Density-Based Spatial Clustering of Applications with Noise)

Este algoritmo conviene aplicarlo para nuestros datos, ya que trabajamos con una gran cantidad de variables continuas y puede identificar clústeres de diferentes formas y tamaños.

Utiliza un modelo matemático de densidad para identificar clústeres en un conjunto de datos. El modelo matemático se basa en la definición de densidad en el espacio de características, y el algoritmo trata de identificar regiones con alta densidad en el espacio de características que corresponden a los clústeres.

Funciona de la siguiente manera: se define un radio epsilón y un número mínimo de puntos que deben estar dentro de un radio epsilón alrededor de un punto para ser considerado una región densa. Cualquier punto que se encuentre en una región densa se considera un punto de núcleo, y los clústeres se identifican conectando puntos de núcleo que están dentro de un radio epsilón uno del otro.

## Métricas para evaluar la calidad de los clústeres 

Los índices de Calinski-Harabasz y Davies-Bouldin son métricas comúnmente utilizadas para evaluar la calidad de los clústeres generados por algoritmos de clustering. Ambos índices se basan en la idea de medir la separación entre los clústeres y la homogeneidad dentro de los clústeres.

Sin embargo, cuando se trata de variables continuas, el índice de **Calinski-Harabasz** suele ser el más adecuado. 

También conocido como el índice de varianza explicada, mide la razón entre la suma de las varianzas intra-clúster y la suma de las varianzas inter-clúster. Un valor más alto de este índice indica una mejor separación entre los clústeres y una mayor homogeneidad dentro de los clústeres.

## Tarea (8 puntos)
- Aplicar al menos un algoritmo no supervisado a tus datos para encontrar estructuras subyacentes
- Elegir alguna métrica para determinar número de grupos, usarla y discutirla
- Busca alguna revista científica que publique trabajos relacionados con el tuyo
- Crea artículo mediante Latex con base en los lineamientos de la revista elegida y redacta ahí tus resultados, discusiones y bibliografía
- Sube el código de tu tarea, los archivos de Latex y el PDF del artículo en tu repositorio, claramente diferenciados